Задача:\
Построить модель машинного обучения, предсказывающую траты у посетителей магазина.\
План решения:
1. Загрузите данные и подготовьте их:
     - Сделайте из текстового признака, обозначающего пол, бинарный флаг 0/1
     - Переименуйте признаки так, чтобы в названиях не было пробелов.
2. Доработайте функцию построения сплита (разработанную в рамках скринкаста):
     - Занесите расчет хаотичности целевого признака в выборке в саму функцию построения сплита;
     - Уберите промежуточные выводы;
     - В качестве результата отработки функции возвращайте пару “признак”+”значение для сплита”, при которых достигается наилучший прирост информации. Если таких не нашлось (если выборка однородна с точки зрения всех признаков и ее невозможно разделить), возвращайте, например, None;
     - Сделаем функцию похожей на библиотечную: пусть она принимает на вход отдельно датафрейм с признаками (назовите параметр X ) и столбец со значениями целевого признака (назовите y). Доработайте код функции под новые входные данные. Подсказка: чтобы понять, какие значения целевого признака будут принимать объекты, попавшие в левую/правую часть подвыборки, создайте маску из столбца с фичей и примените ее к столбца с целевым признаком. Чтобы получить маску, к столбцу в признаком, по которому происходит деление примените операцию сравнения со значением, по которому хотите выполнить сплит. Получится столбец из True/False длины, равной количеству элементов в исходной выборке, подлежащей сплиту. Примените полученную маску к столбцу с целевыми значениями target[mask]. В итоге останутся только значения целевой переменной для наблюдений, которым в маске соответствовало значение True.
3. Реализуйте функцию построения решающего дерева fit_regression_tree, допускающую ограничения на максимальную глубину дерева, а также минимальное число наблюдений в выборке, подлежащей сплиту. В результате запуска функции должны быть выведены правила, по которым должен происходить сплит, а в листьях - предикт (то есть среднее значение целевого признака на подвыборке обучающего датафйрема, попавшей в этот лист). Бинарное дерево строится рекурсивной, поэтому функция его простроения тоже будет рекурсивной:
     - Реализуйте часть для выхода из рекурсии. Здесь должна выполняться проверка на достижение максимально допустимой глубины дерева/достаточное количество наблюдений в выборке для выполнения очередного сплита/возможность построения сплита. Невозможно построить сплит, если все объекты в выборке одинаковы с точки зрения значений фичей. Если одно из этих условий выполняется, выводите предикт в следующем формате:
          ```
          "| " * <текущая глубина дерева>"|---" value: <предикт с 2 знаками после запятой>'
          ```
     - Реализуйте альтернативный путь для дальнейшего построения дерева:
          - Выведите правило, по которому наблюдения “идут влево” в формате:
          ```
            "| " * <текущая глубина дерева>"|---" <фича для сплита> <= <значение для сплита>
          ```
          - Постройте сплит “пошедшей влево” подвыборки
          -   Выведите правило, по которому наблюдения “идут вправо” в формате:
          ```
            "| " * <текущая глубина дерева>"|---" <фича для сплита> > <значение для сплита>
          ```
          - Постройте сплит “пошедшей вправо” подвыборки.

4. Проверьте корректность реализованного алгоритма
     - Убедитесь, что без ошибок строится дерево без ограничений (то есть до последнего возможного сплита)
     - Убедитесь, что без ошибок отрабатывает алгоритм для построения дерева с ограничением на глубину. Выберите максимально допустимую глубину, равную 2. Также постройте дерево с помощью библиотечного алгоритма (DecisionTreeRegressor) на тех же входных данных. Выведите его с помощью функции export_text и сравните построенные деревья. Они должны совпадать.
     - Убедитесь, что без ошибок отрабатывает алгоритм для построения дерева с ограничением на минимальное количество элементов в выборке, над которой производится в сплит. Также сравните построенное дерево с деревом, построенным с помощью библиотечной реализации, при ограничении min_samples_split=70.

In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor

##### 1. Загрузите данные и подготовьте их:


In [2]:
try:
    df = pd.read_csv('Mall_Customers.csv') 
except FileNotFoundError:
    print("You have to load the file to the directory before opening it.")

In [3]:
df.head()

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
CustomerID,200.0,100.50,57.879185,1.0,50.75,100.5,150.25,200.0
Age,200.0,38.85,13.969007,18.0,28.75,36.0,49.00,70.0
Annual Income (k$),200.0,60.56,26.264721,15.0,41.50,61.5,78.00,137.0
Spending Score (1-100),200.0,50.20,25.823522,1.0,34.75,50.0,73.00,99.0


In [5]:
df.nunique()

CustomerID                200
Genre                       2
Age                        51
Annual Income (k$)         64
Spending Score (1-100)     84
dtype: int64

In [6]:
df.isna().sum()

CustomerID                0
Genre                     0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

In [7]:
print(f'Количество дубликатов: {df[df.duplicated()].shape[0]}')

Количество дубликатов: 0


Удалим столбец CustomerID, так как он не несет смысла для анализа данных.

In [8]:
del df['CustomerID']

1.1 Сделайте из текстового признака, обозначающего пол, бинарный флаг 0/1

In [9]:
df["Genre"] = np.select(
    [df["Genre"] == "Male", df["Genre"] == "Female"], [0, 1], default=df["Genre"]
)
df.Genre.value_counts()

Genre
1    112
0     88
Name: count, dtype: int64

1.2 Переименуйте признаки так, чтобы в названиях не было пробелов.

In [10]:
df = df.rename(columns=lambda col: col.split("(")[0].rstrip().lower().replace(" ", "_"))
df.columns

Index(['genre', 'age', 'annual_income', 'spending_score'], dtype='object')

##### 2. Доработайте функцию построения сплита (разработанную в рамках скринкаста)


2.1. Занесите расчет хаотичности целевого признака в выборке в саму функцию построения сплита.\
2.2. Уберите промежуточные выводы.\
2.3. В качестве результата отработки функции возвращайте пару “признак”+”значение для сплита”, при которых достигается наилучший прирост информации. Если таких не нашлось (если выборка однородна с точки зрения всех признаков и ее невозможно разделить), возвращайте, например, None.\
2.4 Сделаем функцию похожей на библиотечную: пусть она принимает на вход отдельно датафрейм с признаками (назовите параметр X ) и столбец со значениями целевого признака (назовите y). Доработайте код функции под новые входные данные.

In [11]:
X = df.drop('spending_score', axis=1)
y = df['spending_score']

In [12]:
def calc_options(X, y):
    max_information_gain = 0
    best_feature_split = None

    def calc_squared_error(values) -> float:
        mean_value = values.mean()
        res = sum([(v - mean_value) ** 2 for v in values]) / len(values)
        return res

    for feature in X.columns.to_list():
        init_criterion_value = calc_squared_error(y)
        diff_feature_values = sorted(np.unique(X[feature]))


        for i in range(len(diff_feature_values) - 1):
            information_gain = init_criterion_value

            split_value = (diff_feature_values[i] + diff_feature_values[i + 1]) / 2

            values_in_group = y[X[feature] <= split_value]
            criterion_value_for_group = calc_squared_error(values_in_group)
            ratio_for_group = len(values_in_group) / len(y)
            information_gain -= criterion_value_for_group * ratio_for_group

            values_in_group = y[X[feature] > split_value]
            criterion_value_for_group = calc_squared_error(values_in_group)
            ratio_for_group = len(values_in_group) / len(y)
            information_gain -= criterion_value_for_group * ratio_for_group

            if max_information_gain <= information_gain:
                max_information_gain = information_gain
                best_feature_split = feature, split_value

    return best_feature_split

In [13]:
best_feature_split, max_information_gain = calc_options(X, y)
print(f'BEST feature_split: {best_feature_split} with information gain = {max_information_gain}')

BEST feature_split: age with information gain = 39.5


3. Реализуйте функцию построения решающего дерева fit_regression_tree, допускающую ограничения на максимальную глубину дерева, а также минимальное число наблюдений в выборке, подлежащей сплиту.

In [14]:
def fit_regression_tree(X, y, max_depth=None, min_samples_split=1, depth = 0):
    current_samples = y.size
    if (len(X.drop_duplicates()) == 1) or max_depth == depth or min_samples_split > current_samples:
        print(f' {"|  " * depth}|---" value: [{round(y.mean(), 2)}]')
        return

    feature, threshold = calc_options(X, y)
    mask = X[feature] <= threshold
    left_X, left_y = X[mask], y[mask]
    rigth_X, rigth_y = X[~mask], y[~mask]
    print(f' {"|  "* depth}|---" {feature} <= {threshold}')
    left_tree = fit_regression_tree(left_X, left_y, max_depth, min_samples_split, depth=depth+1)
    print(f' {"|  "* depth}|---" {feature} <= {threshold}')
    rigth_tree = fit_regression_tree(rigth_X, rigth_y , max_depth, min_samples_split, depth=depth+1)

##### 4. Проверьте корректность реализованного алгоритма.

4.1. Убедитесь, что без ошибок строится дерево без ограничений (то есть до последнего возможного сплита)

In [15]:
fit_regression_tree(X, y)

 |---" age <= 39.5
 |  |---" age <= 20.5
 |  |  |---" annual_income <= 69.0
 |  |  |  |---" annual_income <= 18.5
 |  |  |  |  |---" annual_income <= 15.5
 |  |  |  |  |  |---" value: [39.0]
 |  |  |  |  |---" annual_income <= 15.5
 |  |  |  |  |  |---" value: [6.0]
 |  |  |  |---" annual_income <= 18.5
 |  |  |  |  |---" annual_income <= 41.5
 |  |  |  |  |  |---" age <= 19.0
 |  |  |  |  |  |  |---" value: [92.0]
 |  |  |  |  |  |---" age <= 19.0
 |  |  |  |  |  |  |---" annual_income <= 29.0
 |  |  |  |  |  |  |  |---" value: [66.0]
 |  |  |  |  |  |  |---" annual_income <= 29.0
 |  |  |  |  |  |  |  |---" value: [75.0]
 |  |  |  |  |---" annual_income <= 41.5
 |  |  |  |  |  |---" annual_income <= 53.5
 |  |  |  |  |  |  |---" annual_income <= 47.0
 |  |  |  |  |  |  |  |---" value: [55.0]
 |  |  |  |  |  |  |---" annual_income <= 47.0
 |  |  |  |  |  |  |  |---" age <= 18.5
 |  |  |  |  |  |  |  |  |---" value: [59.0]
 |  |  |  |  |  |  |  |---" age <= 18.5
 |  |  |  |  |  |  |  |

  4.2. Убедитесь, что без ошибок отрабатывает алгоритм для построения дерева с ограничением на глубину. Выберите максимально допустимую глубину, равную 2. Также постройте дерево с помощью библиотечного алгоритма (DecisionTreeRegressor) на тех же входных данных. Выведите его с помощью функции export_text и сравните построенные деревья. Они должны совпадать.

In [16]:
fit_regression_tree(X, y, max_depth=2)

 |---" age <= 39.5
 |  |---" age <= 20.5
 |  |  |---" value: [44.65]
 |  |---" age <= 20.5
 |  |  |---" value: [62.53]
 |---" age <= 39.5
 |  |---" annual_income <= 72.0
 |  |  |---" value: [41.77]
 |  |---" annual_income <= 72.0
 |  |  |---" value: [19.79]


In [17]:
model = DecisionTreeRegressor(max_depth=2)
model.fit(X, y)
print(tree.export_text(model, feature_names=X.columns.to_list()))

|--- age <= 39.50
|   |--- age <= 20.50
|   |   |--- value: [44.65]
|   |--- age >  20.50
|   |   |--- value: [62.53]
|--- age >  39.50
|   |--- annual_income <= 72.00
|   |   |--- value: [41.77]
|   |--- annual_income >  72.00
|   |   |--- value: [19.79]



4.3. Убедитесь, что без ошибок отрабатывает алгоритм для построения дерева с ограничением на минимальное количество элементов в выборке, над которой производится в сплит. Также сравните построенное дерево с деревом, построенным с помощью библиотечной реализации, при ограничении min_samples_split=70.

In [18]:
fit_regression_tree(X, y, min_samples_split=70)

 |---" age <= 39.5
 |  |---" age <= 20.5
 |  |  |---" value: [44.65]
 |  |---" age <= 20.5
 |  |  |---" age <= 32.5
 |  |  |  |---" value: [66.59]
 |  |  |---" age <= 32.5
 |  |  |  |---" value: [55.09]
 |---" age <= 39.5
 |  |---" annual_income <= 72.0
 |  |  |---" value: [41.77]
 |  |---" annual_income <= 72.0
 |  |  |---" value: [19.79]


In [19]:
model = DecisionTreeRegressor(min_samples_split=70)
model.fit(X, y)
print(tree.export_text(model, feature_names=X.columns.to_list()))

|--- age <= 39.50
|   |--- age <= 20.50
|   |   |--- value: [44.65]
|   |--- age >  20.50
|   |   |--- age <= 32.50
|   |   |   |--- value: [66.59]
|   |   |--- age >  32.50
|   |   |   |--- value: [55.09]
|--- age >  39.50
|   |--- annual_income <= 72.00
|   |   |--- value: [41.77]
|   |--- annual_income >  72.00
|   |   |--- value: [19.79]



Для тренировки написала класс MyRegressionTree.

In [20]:
class MyRegressionTree:
    """
    It's a custom class if Regression Tree.

    Parametes:
    ----------
    min_samples_split : int, default=2

    max_depth : int, default=None
    """

    def __init__(self, max_depth=None, min_samples_split=1):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    def _calc_squared_error(self, values) -> float:
        """
        Function count squared error
        """
        mean_value = values.mean()
        res = sum([(v - mean_value) ** 2 for v in values]) / len(values)
        return res

    def _get_gain_from_group(self, values_in_group, gain):
        """
        Function for ger information gain from group of values.
        """
        criterion_value_for_group = self._calc_squared_error(values_in_group)
        ratio_for_group = len(values_in_group) / len(y)
        gain -= criterion_value_for_group * ratio_for_group
        return gain

    def _split(self, X, y, feature, threshold):
        feature_values = X[feature]
        mask = feature_values <= threshold
        left_indexes = X[mask].index
        right_indexes = X[~mask].index
        return left_indexes, right_indexes

    def _get_best_split(self, X, y):
        """
        Function for get the best split into groups.
        """
        max_information_gain = 0

        for feature in X.columns.to_list():
            init_criterion_value = self._calc_squared_error(y)
            unique_values = sorted(np.unique(X[feature]))

            for i in range(len(unique_values) - 1):
                info_gain = init_criterion_value
                threshold = (unique_values[i] + unique_values[i + 1]) / 2
                left_indexes, right_indexes = self._split(X, y, feature, threshold)
                values_left, values_right = (y.loc[left_indexes], y.loc[right_indexes])
                info_gain = self._get_gain_from_group(values_left, info_gain)
                info_gain = self._get_gain_from_group(values_right, info_gain)

                if max_information_gain <= info_gain:
                    max_information_gain = info_gain
                    best_feature_split = feature, threshold

        return best_feature_split

    def _stop_recursion(self, X, y, depth):
        """
        Conditions for stop recursion.
        """
        return (
            len(X.drop_duplicates()) == 1,
            self.max_depth == depth,
            self.min_samples_split > y.size,
        )

    def _fit_regression_tree(self, X, y, depth=0):
        """
        Function for fit data, using recursion.
        """
        if any(self._stop_recursion(X, y, depth)):
            print(f' {"|  " * depth}|---" value: [{round(y.mean(), 2)}]')
            return

        feature, threshold = self._get_best_split(X, y)
        left_indexes, right_indexes = self._split(X, y, feature, threshold)
        left_X, right_X = X.loc[left_indexes], X.loc[right_indexes]
        left_y, right_y = y.loc[left_indexes], y.loc[right_indexes]
        print(f' {"|  "* depth}|---" {feature} <= {threshold}')
        left_tree = self._fit_regression_tree(left_X, left_y, depth=depth + 1)
        print(f' {"|  "* depth}|---" {feature} <= {threshold}')
        rigth_tree = self._fit_regression_tree(right_X, right_y, depth=depth + 1)

    def fit(self, X, y):
        """
        Function for fit data.
        """
        self._fit_regression_tree(X, y)

In [21]:
my_model = MyRegressionTree()
my_model.fit(X, y)

 |---" age <= 39.5
 |  |---" age <= 20.5
 |  |  |---" annual_income <= 69.0
 |  |  |  |---" annual_income <= 18.5
 |  |  |  |  |---" annual_income <= 15.5
 |  |  |  |  |  |---" value: [39.0]
 |  |  |  |  |---" annual_income <= 15.5
 |  |  |  |  |  |---" value: [6.0]
 |  |  |  |---" annual_income <= 18.5
 |  |  |  |  |---" annual_income <= 41.5
 |  |  |  |  |  |---" age <= 19.0
 |  |  |  |  |  |  |---" value: [92.0]
 |  |  |  |  |  |---" age <= 19.0
 |  |  |  |  |  |  |---" annual_income <= 29.0
 |  |  |  |  |  |  |  |---" value: [66.0]
 |  |  |  |  |  |  |---" annual_income <= 29.0
 |  |  |  |  |  |  |  |---" value: [75.0]
 |  |  |  |  |---" annual_income <= 41.5
 |  |  |  |  |  |---" annual_income <= 53.5
 |  |  |  |  |  |  |---" annual_income <= 47.0
 |  |  |  |  |  |  |  |---" value: [55.0]
 |  |  |  |  |  |  |---" annual_income <= 47.0
 |  |  |  |  |  |  |  |---" age <= 18.5
 |  |  |  |  |  |  |  |  |---" value: [59.0]
 |  |  |  |  |  |  |  |---" age <= 18.5
 |  |  |  |  |  |  |  |

In [22]:
my_model = MyRegressionTree(min_samples_split=70)
my_model.fit(X, y)

 |---" age <= 39.5
 |  |---" age <= 20.5
 |  |  |---" value: [44.65]
 |  |---" age <= 20.5
 |  |  |---" age <= 32.5
 |  |  |  |---" value: [66.59]
 |  |  |---" age <= 32.5
 |  |  |  |---" value: [55.09]
 |---" age <= 39.5
 |  |---" annual_income <= 72.0
 |  |  |---" value: [41.77]
 |  |---" annual_income <= 72.0
 |  |  |---" value: [19.79]


In [23]:
my_model = MyRegressionTree(max_depth=2)
my_model.fit(X, y)

 |---" age <= 39.5


 |  |---" age <= 20.5
 |  |  |---" value: [44.65]
 |  |---" age <= 20.5
 |  |  |---" value: [62.53]
 |---" age <= 39.5
 |  |---" annual_income <= 72.0
 |  |  |---" value: [41.77]
 |  |---" annual_income <= 72.0
 |  |  |---" value: [19.79]


*Вывод*:\
Получившаяся кастомная модель работает на уровне библиотечной, позволяет работать с настройкой глубины и размером выборки для сплита.